## Benchmarking TuRBO Bayesian Optimization
In this tutorial we demonstrate the use of Xopt to preform Bayesian Optimization on
the 20D Ackley test function.

## Define the test problem
Here we define a simple optimization problem, where we attempt to minimize the sin
function in the domian [0,2*pi]. Note that the function used to evaluate the
objective function takes a dictionary as input and returns a dictionary as the output.

In [1]:
from xopt.resources.test_functions.sphere_20 import vocs, evaluate_sphere


## Create Xopt objects
Create the evaluator to evaluate our test function and create a generator that uses
the Upper Confidence Bound acqusition function to perform Bayesian Optimization.

In [2]:
from xopt.evaluator import Evaluator
from xopt.generators.bayesian import TuRBOUpperConfidenceBoundGenerator
from xopt import Xopt

evaluator = Evaluator(function=evaluate_sphere)
generator = TuRBOUpperConfidenceBoundGenerator(vocs)
generator.options.n_initial = 5
generator.options.optim.num_restarts=20
X = Xopt(evaluator=evaluator, generator=generator, vocs=vocs)

## Generate and evaluate initial points
To begin optimization, we must generate some random initial data points. The first call
to `X.step()` will generate and evaluate a number of randomly points specified by the
 generator. Note that if we add data to xopt before calling `X.step()` by assigning
 the data to `X.data`, calls to `X.step()` will ignore the random generation and
 proceed to generating points via Bayesian optimization.

In [3]:
# evaluate random initial points
X.step()

# inspect the gathered data
X.data

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x13,x14,x15,x16,x17,x18,x19,f,xopt_runtime,xopt_error
1,0.062995,-0.062279,0.544839,-0.503431,0.821776,0.629965,-0.474384,-0.033765,-0.387065,-0.940502,...,-0.615613,-0.160023,-0.488284,-0.947566,0.328403,-0.366973,0.738352,6.1677294,0.000156,False
2,-0.561243,0.512332,-0.632823,0.350470,-0.523234,0.384745,-0.785675,-0.149247,-0.831706,-0.579351,...,-0.314386,-0.732740,0.657981,-0.765338,0.152458,0.349693,0.601206,6.9899263,0.000030,False
3,-0.901420,-0.238834,0.060313,0.320931,0.515013,-0.946428,0.467778,0.294698,-0.681843,0.671658,...,0.524845,0.225160,0.236900,-0.977615,0.674749,-0.572095,-0.380031,6.0905423,0.000025,False
4,-0.368522,-0.159474,0.759957,0.700507,-0.905647,0.287221,-0.391568,-0.171503,-0.177241,0.405613,...,-0.494040,0.322425,-0.006682,0.390137,-0.551452,-0.388566,0.240009,5.3742104,0.000034,False
5,0.551577,-0.984087,-0.566301,0.200010,0.372020,-0.720478,-0.538366,-0.699903,-0.799647,-0.825138,...,-0.529741,-0.016578,0.406307,0.833492,0.959548,-0.175618,-0.134072,7.0636697,0.000024,False


In [4]:
# determine trust region from gathered data
generator.train_model()
generator.get_trust_region()

tensor([[-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.]], dtype=torch.float64)

## Do bayesian optimization steps
To perform optimization we simply call `X.step()` in a loop. This allows us to do
intermediate tasks in between optimization steps, such as examining the model and
acquisition function at each step (as we demonstrate here).

In [5]:
import torch

print(X.generator.turbo_state.failure_tolerance)
for i in range(200):
    print(f"{X.generator.turbo_state.length}, "
          f"sc:{X.generator.turbo_state.success_counter}, "
          f"fc:{X.generator.turbo_state.failure_counter},"
          f"best_val: {X.generator.turbo_state.best_value}"
          )
    # do the optimization step
    X.step()


20
0.5, sc:0, fc:0,best_val: inf
0.5, sc:1, fc:0,best_val: 7.063669681549072
0.5, sc:2, fc:0,best_val: 5.595542907714844
0.5, sc:3, fc:0,best_val: 5.253245830535889
0.5, sc:4, fc:0,best_val: 3.20282244682312
0.5, sc:0, fc:1,best_val: 3.20282244682312
0.5, sc:1, fc:0,best_val: 2.684757947921753
0.5, sc:0, fc:1,best_val: 2.684757947921753
0.5, sc:0, fc:2,best_val: 2.684757947921753
0.5, sc:0, fc:3,best_val: 2.684757947921753
0.5, sc:0, fc:4,best_val: 2.684757947921753
0.5, sc:0, fc:5,best_val: 2.684757947921753
0.5, sc:0, fc:6,best_val: 2.684757947921753
0.5, sc:0, fc:7,best_val: 2.684757947921753
0.5, sc:0, fc:8,best_val: 2.684757947921753
0.5, sc:0, fc:9,best_val: 2.684757947921753
0.5, sc:0, fc:10,best_val: 2.684757947921753
0.5, sc:0, fc:11,best_val: 2.684757947921753
0.5, sc:0, fc:12,best_val: 2.684757947921753
0.5, sc:0, fc:13,best_val: 2.684757947921753
0.5, sc:0, fc:14,best_val: 2.684757947921753
0.5, sc:0, fc:15,best_val: 2.684757947921753
0.5, sc:0, fc:16,best_val: 2.6847579479

In [14]:
# access the collected data
X.data

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x13,x14,x15,x16,x17,x18,x19,f,xopt_runtime,xopt_error
1,0.062995,-0.062279,0.544839,-0.503431,0.821776,0.629965,-0.474384,-0.033765,-0.387065,-0.940502,...,-0.615613,-0.160023,-0.488284,-0.947566,0.328403,-0.366973,0.738352,6.1677294,0.000156,False
2,-0.561243,0.512332,-0.632823,0.350470,-0.523234,0.384745,-0.785675,-0.149247,-0.831706,-0.579351,...,-0.314386,-0.732740,0.657981,-0.765338,0.152458,0.349693,0.601206,6.9899263,0.000030,False
3,-0.901420,-0.238834,0.060313,0.320931,0.515013,-0.946428,0.467778,0.294698,-0.681843,0.671658,...,0.524845,0.225160,0.236900,-0.977615,0.674749,-0.572095,-0.380031,6.0905423,0.000025,False
4,-0.368522,-0.159474,0.759957,0.700507,-0.905647,0.287221,-0.391568,-0.171503,-0.177241,0.405613,...,-0.494040,0.322425,-0.006682,0.390137,-0.551452,-0.388566,0.240009,5.3742104,0.000034,False
5,0.551577,-0.984087,-0.566301,0.200010,0.372020,-0.720478,-0.538366,-0.699903,-0.799647,-0.825138,...,-0.529741,-0.016578,0.406307,0.833492,0.959548,-0.175618,-0.134072,7.0636697,0.000024,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,-0.027026,0.035315,-0.030245,0.030285,0.020834,-0.016995,0.009642,0.018554,0.019674,-0.018592,...,-0.021082,0.028225,-0.024120,-0.003468,-0.010130,0.020217,-0.041383,0.012211022,0.000061,False
202,0.006212,0.035013,0.012527,-0.003776,0.022976,0.016143,-0.014786,0.018250,-0.009620,0.010238,...,0.002984,-0.004988,-0.037240,-0.003862,-0.016839,-0.001926,-0.041088,0.008816749,0.000055,False
203,-0.026574,0.034857,0.012320,0.003896,0.022817,0.015985,-0.023450,-0.013961,0.019353,0.010468,...,-0.029400,0.027764,-0.024597,-0.004058,-0.011925,0.025997,-0.009412,0.0097027,0.000047,False
204,-0.017086,0.034640,-0.022766,0.029541,-0.007904,0.015767,-0.017971,0.017889,0.019539,0.012732,...,0.002497,-0.004412,-0.043894,-0.004313,-0.026123,-0.003168,-0.040722,0.010275573,0.000067,False


## Getting the trust region

In [13]:
X.generator.get_trust_region()

tensor([[-0.0262,  0.0031, -0.0092,  0.0048, -0.0669, -0.0380, -0.0045, -0.0580,
         -0.0433, -0.0313, -0.0030, -0.0406, -0.0292, -0.0051, -0.0098, -0.0162,
         -0.0230, -0.0136, -0.0090, -0.0175],
        [ 0.0057,  0.0345,  0.0256,  0.0382, -0.0311,  0.0023,  0.0274, -0.0250,
         -0.0045,  0.0010,  0.0321, -0.0098,  0.0118,  0.0294,  0.0224,  0.0156,
          0.0122,  0.0177,  0.0301,  0.0247]], dtype=torch.float64)

## Customizing optimization
Each generator has a set of options that can be modified to effect optimization behavior

In [8]:
X.generator.options.dict()

{'optim': {'num_restarts': 20,
  'raw_samples': 20,
  'sequential': True,
  'use_nearby_initial_points': False,
  'max_travel_distances': None},
 'acq': {'proximal_lengthscales': None,
  'use_transformed_proximal_weights': True,
  'monte_carlo_samples': 128,
  'beta': 2.0},
 'model': {'name': 'standard',
  'custom_constructor': None,
  'use_low_noise_prior': True,
  'covar_modules': {},
  'mean_modules': {}},
 'n_initial': 5,
 'use_cuda': False}

In [9]:
# example: add a Gamma(1.0,10.0) prior to the noise hyperparameter to reduce model noise
# (good for optimizing noise-free simulations)
X.generator.options.model.use_low_noise_prior = True

In [16]:
list(X.generator.model.named_parameters())

[('models.0.likelihood.noise_covar.raw_noise',
  Parameter containing:
  tensor([-18.9156], dtype=torch.float64, requires_grad=True)),
 ('models.0.mean_module.raw_constant',
  Parameter containing:
  tensor(2.3018, dtype=torch.float64, requires_grad=True)),
 ('models.0.covar_module.raw_outputscale',
  Parameter containing:
  tensor(-1.5870, dtype=torch.float64, requires_grad=True)),
 ('models.0.covar_module.base_kernel.raw_lengthscale',
  Parameter containing:
  tensor([[0.2928, 0.2690, 0.4275, 0.3611, 0.4694, 0.6586, 0.2928, 0.3429, 0.5969,
           0.3127, 0.4361, 0.2421, 0.6849, 0.4104, 0.3078, 0.2881, 0.4446, 0.2677,
           0.6057, 0.7308]], dtype=torch.float64, requires_grad=True))]